In [2]:
!pip install tensorflow==1.15


In [4]:
import os

pretrained_model_dir = os.path.join(os.getcwd(), "models")
os.makedir(pretrained_model_dir, exist_ok=True)

restyle_dir = os.path.join("/content", "restyle")
stylegan_ada_dir = os.path.join("/content", "stylegan_ada")
stylegan_nada_dir = os.path.join("/content", "stylegan_nada")

output_dir = os.path.join("/content", "output")

output_model_dir = os.path.join(output_dir, "models")
output_image_dir = os.path.join(output_dir, "images")

PermissionError: [Errno 13] Permission denied: '/content'

In [5]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/thomdc/code/Users/thom.d/stylegan2-experiments
